In [1]:
# For reading in Json file
import json

# general packages
import pandas as pd
import numpy as np
import time

# For selecting period of valuation table
from dateutil.relativedelta import relativedelta

In [2]:
# Load the autoreload extension
%load_ext autoreload
%autoreload 1

# Import functions for scraping the data
%aimport get_data
import get_data as gd

# Import functions for analysing the data
%aimport analyse_portfolio
import analyse_portfolio as ap

In [18]:
# Update the default number of rows that are displayed
pd.set_option('display.max_rows', 500)

In [ ]:
gd.create_company_ticker_to_name_map()

In [32]:
transactions = gd.read_in_raw_transactions_data()
transactions

,date,stock_ticker,exchange_ticker,num_shares,share_price,exchange_rate
0,2020-06-17,GOLD,NSY,3,24.2800,1.12220
1,2020-06-25,GOLD,NSY,5,25.4000,1.12020
2,2020-07-13,GOLD,NSY,4,26.9500,1.13490
3,2020-07-20,GOLD,NSY,-4,28.1350,1.14520
4,2020-08-07,GOLD,NSY,30,28.8200,1.17690
5,2020-08-11,GOLD,NSY,20,26.6700,1.17670
6,2020-08-11,GOLD,NSY,20,26.6700,1.17580
7,2020-08-27,GOLD,NSY,7,28.3100,1.17710
8,2020-09-04,GOLD,NSY,31,28.3000,1.17790
9,2020-09-04,GOLD,NSY,70,28.2200,1.17820


In [57]:
transactions = gd.create_company_name_col_using_ticker(transactions)
transactions = gd.create_exchange_name_nd_curency_cols_using_ticker(transactions)

The following stocks did not have an associated company name:
|     | stock_ticker   | exchange_ticker   |
|----:|:---------------|:------------------|
|  45 | SVE            | TSV               |
|  49 | NOVR           | TSV               |
|  52 | MMG            | TSV               |
|  53 | RSLV           | TSV               |
|  59 | LGD            | TOR               |
|  61 | SSV            | TSV               |
| 101 | ELEC           | TSV               |


In [20]:
# Ensure the table is sorted by ticker and then by date
transactions.sort_values(["stock_ticker", "date"], inplace=True, ignore_index=True)
transactions

,date,stock_ticker,exchange_ticker,num_shares,share_price,exchange_rate,company_name,currency,exchange_name
0,2021-06-14,AGI,NSY,28,8.7000,1.21160,Alamos Gold,USD,New York Stock Exchange
1,2021-06-17,ASM,ASE,200,1.2100,1.19130,Avino Silver & Gold Mines,USD,NYSE MKT - Amex
2,2021-06-14,AUY,NSY,49,5.0300,1.21150,Yamana Gold,USD,New York Stock Exchange
3,2021-01-27,AXU,ASE,50,2.5000,1.20870,Alexco Resource,USD,NYSE MKT - Amex
4,2021-03-31,AXU,ASE,50,2.3900,1.17160,Alexco Resource,USD,NYSE MKT - Amex
5,2021-04-26,AXU,ASE,50,2.7400,1.20650,Alexco Resource,USD,NYSE MKT - Amex
6,2021-07-16,AXU,ASE,110,2.1000,1.17980,Alexco Resource,USD,NYSE MKT - Amex
7,2021-08-17,AXU,ASE,200,1.6800,1.17030,Alexco Resource,USD,NYSE MKT - Amex
8,2021-05-21,BABA,NSY,1,212.0000,1.21660,Alibaba Group,USD,New York Stock Exchange
9,2021-08-27,BABA,NSY,3,1.5958,1.17290,Alibaba Group,USD,New York Stock Exchange


In [6]:
unique_tickers_list = list(transactions["stock_ticker"].drop_duplicates())
ticker_start_date_list = [str(min(transactions[transactions["stock_ticker"] == ticker]["date"]))[:10] for ticker in unique_tickers_list]
gd.create_table_of_historic_stock_prices(unique_tickers_list, ticker_start_date_list)

- LGD: No data found for this date range, symbol may be delisted
LGD not found
- MMG: No data found for this date range, symbol may be delisted
MMG not found
- NOVR: No data found, symbol may be delisted
NOVR not found
- RSLV: No data found, symbol may be delisted
RSLV not found
- SSV: No data found for this date range, symbol may be delisted
SSV not found
- SVE: No data found for this date range, symbol may be delisted
SVE not found


In [7]:
unique_currency_list = list(transactions["currency"].drop_duplicates())
start_date = min(transactions["date"]).date()
base_currency = 'EUR'
gd.create_exchange_rate_to_date_map(base_currency, unique_currency_list, start_date)

In [8]:
stock_data = ap.get_specific_stock(transactions, "GOLD", "NSY")

In [9]:
stock_data.dtypes

date                        datetime64[ns]
stock_ticker                        object
exchange_ticker                     object
num_shares                           int64
share_price                        float64
exchange_rate                      float64
company_name                        object
currency                            object
exchange_name                       object
share_cost_in_$                    float64
currency_exchange_fee              float64
fixed_transaction_fee              float64
variable_transaction_fee           float64
total_outgoing_in_eur              float64
share_cost_in_eur                  float64
total_shares_to_date                 int64
break_even_price                   float64
dtype: object

In [35]:
daily_account_balance_df = ap.calculate_daily_account_balance(stock_data)

In [50]:
# {"day": relativedelta(days=1),
#  "week": relativedelta(weeks=1),
#  "month": relativedelta(months=1),
#  "quarter": relativedelta(months=3),
#  "year": relativedelta(years=1),
# }

base_currency = "EUR"
ap.create_valuation_table(stock_data, relativedelta(weeks=1), base_currency)

TypeError: unsupported operand type(s) for +: 'int' and 'str'